In [2]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-11-18 21:37:18--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-11-18 21:37:19 (6.03 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [6]:
# https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz



In [8]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [10]:
# Count the number of records (rows) in the dataset.

row = df.count()

print(row)

3093869


In [77]:
# Transform the dataset to fit the tables in the schema file. Be sure the DataFrames match in data type and in column name.

review_id_table_df = df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])

products_df = df.select(['product_id', 'product_title'])

customers_df = df.select(['customer_id'])

vine_table_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine'])

# Review Table Transformations

In [78]:
review_id_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: string (nullable = true)



In [79]:
from pyspark.sql.functions import *


review_id_table_df = review_id_table_df.withColumn("review_date",to_date("review_date","yyyy-mm-dd").alias("review_date"))

In [80]:
row = review_id_table_df.count()

print(row)

3093869


In [81]:
review_id_table_df = review_id_table_df.filter(review_id_table_df["review_id"].isNotNull())

In [82]:
row = review_id_table_df.count()

print(row)

3093869


In [83]:
review_id_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



# Product Table Transformations

In [84]:
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [64]:
products_df = products_df.filter(products_df["product_id"].isNotNull())

In [86]:
products_df = products_df.dropDuplicates(["product_id"])

In [87]:
row = products_df.count()

print(row)

185852


# Customer Table Transformations

In [45]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)



In [46]:
customers_df = customers_df.groupBy('customer_id').count()

In [48]:
customers_df = customers_df.withColumnRenamed('count','customer_count')

In [54]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

customers_df = customers_df.withColumn("customer_count", col("customer_count").cast(IntegerType()))


In [65]:
customers_df = customers_df.filter(customers_df["customer_id"].isNotNull())

In [55]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [90]:
customers_df = customers_df.dropDuplicates(["customer_id"])

# Vine Table Transformations

In [56]:
vine_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



# Load into RDS 

In [72]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<endpoint>:5432/<database>"
config = {"user":"<user_name>",
          "password": "<password>",
          "driver":"org.postgresql.Driver"}

In [73]:
review_id_table_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [88]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [91]:
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [92]:
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)